In [8]:
import pandas as pd
import glob
import math
import numpy as np
import copy

In [2]:
def evaluate(submission,groundtruth):
    ap_sum = 0.0
    first_pred_acc_sum = 0.0
    counter = 0
    for sub, tru in zip(submission, groundtruth):
        if len(sub) != len(tru):
            raise Exception('Line {} should contain {} predictions, but instead contains '
                            '{}'.format(counter+1,len(tru),len(sub)))
        ap_sum += ave_pre(sub,tru,counter)
        first_pred_acc_sum += sub[0] == tru[0]
        counter+=1
    ap = ap_sum/counter
    first_pred_acc = first_pred_acc_sum/counter
    return ap,first_pred_acc

In [21]:
def ave_pre(submission,groundtruth,counter):
    s = 0.0
    t = 0.0
    c = 1.0
    for x, y in zip(submission, groundtruth):
#         print('x',x)
#         print('y',y)
        if x != 0 and x != 1:
            raise Exception('Invalid prediction in line {}, should be 0 or 1'.format(counter))
        if x==y:
            s += 1.0
            t += s / c
#             print('s',s)
#             print('c',c)
#             print('t',t)
        c += 1
#     print('t',t)
    return t/len(groundtruth)

In [5]:
ave_pre([0,0],[0,0],0)

x 0
y 0
s 1.0
c 1.0
t 1.0
x 0
y 0
s 2.0
c 2.0
t 2.0


1.0

In [29]:
l = 10

In [30]:
a = [0] * l

In [31]:
b = [0] * l

In [32]:
b_list = []
for i in range(l):
    c = copy.deepcopy(b)
    c[i] = 1
    b_list.append(c)

In [33]:
b_list

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]

In [34]:
for ex_b in b_list:
    print(ave_pre(ex_b,a,0))

0.7071031746031746
0.7571031746031747
0.7904365079365079
0.8154365079365078
0.8354365079365079
0.8521031746031745
0.866388888888889
0.8788888888888889
0.89
0.9


In [41]:
import os
from os import listdir
from os.path import isfile, join
import csv
import tqdm
import copy
import pandas as pd
import numpy as np
import glob


def mapbool(s):
    return s.lower in ['true','1']

train_set_dir = "/media/data2/Data/wsdm2019/data/training_set/"
test_set_dir = "/media/data2/Data/wsdm2019/data/test_set/"
track_features_dir = "/media/data2/Data/wsdm2019/data/track_features/"

output_dir = "/media/data2/Data/wsdm2019/data/python/"

large_train_set_dir = "/media/data2/Data/wsdm2019/python/data/training_set/"

track_feature_files = sorted(glob.glob(track_features_dir+"*.csv"))

In [42]:
print(track_feature_files)

['/media/data2/Data/wsdm2019/data/track_features/tf_000000000000.csv', '/media/data2/Data/wsdm2019/data/track_features/tf_000000000001.csv']


In [43]:
# for file_dir in track_feature_files:
track_feature_dir_0 = track_feature_files[0]
track_feature_0 = pd.read_csv(track_feature_dir_0)

track_feature_dir_1 = track_feature_files[1]
track_feature_1 = pd.read_csv(track_feature_dir_1)

track_feature_all = track_feature_0.append(track_feature_1)

In [55]:
track_feature_all.columns

Index(['track_id', 'duration', 'release_year', 'us_popularity_estimate',
       'acousticness', 'beat_strength', 'bounciness', 'danceability',
       'dyn_range_mean', 'energy', 'flatness', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mechanism', 'mode', 'organism', 'speechiness',
       'tempo', 'time_signature', 'valence', 'acoustic_vector_0',
       'acoustic_vector_1', 'acoustic_vector_2', 'acoustic_vector_3',
       'acoustic_vector_4', 'acoustic_vector_5', 'acoustic_vector_6',
       'acoustic_vector_7'],
      dtype='object')

In [51]:
pd.set_option('display.max_columns', None)

In [52]:
track_feature_all.describe()

,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,organism,speechiness,tempo,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7
count,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06
mean,2.331915e+02,2.009261e+03,9.557394e+01,3.469715e-01,4.617475e-01,4.791122e-01,5.565486e-01,7.848266e+00,5.918491e-01,9.934531e-01,2.070054e-01,5.272434e+00,2.126515e-01,-9.604988e+00,4.996207e-01,4.618936e-01,1.045310e-01,1.200684e+02,3.874560e+00,4.814764e-01,-1.044815e-01,1.553606e-02,5.953425e-02,5.320778e-02,2.273738e-02,6.459076e-04,-2.493560e-02,-1.594925e-03
std,1.084008e+02,1.103289e+01,2.735566e+00,3.426439e-01,1.717593e-01,1.990264e-01,1.858440e-01,2.680296e+00,2.602815e-01,5.521685e-02,3.447905e-01,3.564020e+00,1.924461e-01,5.732654e+00,2.561809e-01,2.509198e-01,1.385980e-01,3.042757e+01,4.851897e-01,2.725142e-01,4.323750e-01,3.607428e-01,2.804066e-01,3.821163e-01,4.107566e-01,2.118570e-01,3.742747e-01,3.871178e-01
min,3.000095e+01,1.950000e+03,9.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-6.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.174042e+00,-1.185975e+00,-8.858698e-01,-8.994451e-01,-1.081056e+00,-1.063290e+00,-8.022487e-01,-1.178501e+00
25%,1.767467e+02,2.006000e+03,9.338183e+01,2.667680e-02,3.316528e-01,3.206987e-01,4.326980e-01,5.821963e+00,4.098319e-01,9.691408e-01,2.465405e-07,2.000000e+00,9.680310e-02,-1.171000e+01,2.692308e-01,2.532349e-01,3.627243e-02,9.603400e+01,4.000000e+00,2.519543e-01,-4.085117e-01,-1.757177e-01,-1.471524e-01,-2.348342e-01,-3.122829e-01,-9.018810e-02,-3.155564e-01,-2.704680e-01
50%,2.179135e+02,2.013000e+03,9.575417e+01,2.205129e-01,4.593159e-01,4.795972e-01,5.727441e-01,7.519920e+00,6.261150e-01,1.002422e+00,2.161874e-04,5.000000e+00,1.294195e-01,-8.084000e+00,4.854369e-01,4.549237e-01,5.012670e-02,1.199480e+02,4.000000e+00,4.736025e-01,-4.000132e-02,8.460746e-02,1.291028e-01,6.207571e-02,-1.709655e-02,4.277601e-02,-9.838638e-02,-4.027091e-03
75%,2.671733e+02,2.017000e+03,9.792122e+01,6.528423e-01,5.851562e-01,6.339014e-01,6.975901e-01,9.476902e+00,8.073052e-01,1.029172e+00,3.132863e-01,8.000000e+00,2.700362e-01,-5.791000e+00,7.173145e-01,6.480055e-01,1.015356e-01,1.398600e+02,4.000000e+00,7.050039e-01,1.915962e-01,2.832876e-01,2.746523e-01,2.832823e-01,3.724048e-01,1.476096e-01,2.058770e-01,2.218316e-01
max,1.799999e+03,2.019000e+03,1.000000e+02,9.957964e-01,9.999525e-01,9.862775e-01,9.978067e-01,5.261309e+01,9.999841e-01,1.167801e+00,9.999964e-01,1.100000e+01,1.000000e+00,6.275000e+00,1.000000e+00,9.791129e-01,9.705799e-01,2.499870e+02,5.000000e+00,9.999900e-01,1.000457e+00,8.624035e-01,6.509755e-01,1.202726e+00,1.000955e+00,4.130149e-01,1.092165e+00,1.204228e+00


In [53]:
track_feature_all.head()

,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,mode,organism,speechiness,tempo,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7
0,t_2e8f4b71-8a0b-4b9c-b7d8-fb5208e87f9f,326.013336,1971,99.582885,0.716209,0.366495,0.332605,0.439835,5.805774,0.238847,1.010700,6.533861e-01,0,0.769258,-17.094,0.199170,major,0.759699,0.033940,100.370003,4,0.223395,0.146012,-0.706908,0.259496,0.481157,0.238427,-0.098389,-0.254960,-0.227383
1,t_dae2ec0e-ec7b-4b3e-b60c-4a884d0eccb0,147.813324,1963,97.272035,0.839460,0.362212,0.389829,0.507580,6.845427,0.420476,1.000398,3.941550e-09,0,0.085844,-11.295,0.357639,major,0.747436,0.049856,141.334000,4,0.484702,0.039554,-0.539554,0.105141,0.692589,0.226047,-0.468162,0.164389,-0.769024
2,t_cf0164dd-1531-4399-bfa6-dec19cd1fedc,110.400002,1974,99.620384,0.054673,0.495002,0.589378,0.552311,9.361949,0.842938,0.957766,1.041595e-01,0,0.407325,-9.310,0.304721,major,0.493154,0.071753,138.889999,4,0.818441,0.083863,-0.242108,-0.014258,0.096396,0.417641,-0.050576,-0.204757,-0.172563
3,t_0f90acc7-d5c5-4e53-901d-55610fbd090c,237.653336,1988,96.796830,0.042606,0.389634,0.359044,0.585673,6.068578,0.665398,0.947322,1.444963e-05,0,0.251502,-12.159,0.702948,major,0.212197,0.029425,133.139008,4,0.594829,0.192498,0.340039,0.034846,-0.389794,0.518381,0.185008,-0.079907,-0.016978
4,t_36b9ad02-095a-443d-a697-6c7285d9410a,174.600006,1987,97.905891,0.249982,0.513640,0.485435,0.635095,7.198735,0.408715,1.014063,5.266880e-01,0,0.218370,-13.813,0.888889,major,0.193438,0.032178,152.212006,4,0.591289,0.270586,-0.411061,0.165898,0.225652,0.335518,-0.036643,-0.016300,-0.446870


In [90]:
track_feature_copy = track_feature_all.copy(deep=True)

In [85]:
normalize_columns = ['duration', 'release_year', 'us_popularity_estimate',
                    'acousticness', 'beat_strength', 'bounciness', 'danceability',
                    'dyn_range_mean', 'energy', 'flatness', 'instrumentalness', 'key',
                    'liveness', 'loudness', 'mechanism', 'organism', 'speechiness',
                    'tempo', 'time_signature', 'valence']

categorical_columns = ['mode']

In [92]:
# do min-max scaling which is (x - min) / (max - min), if value is bigger than max set to max
for feature in normalize_columns:
    min = track_feature_copy[feature].min()
    max = track_feature_copy[feature].max()
    track_feature_copy[feature] = track_feature_copy[feature].clip(min, max)
    track_feature_copy[feature] = (track_feature_copy[feature] - min) / (max - min)

In [93]:
# do one hot encoding for the categorical variable
for feature in categorical_columns:
    track_feature_copy = pd.concat([track_feature_copy, pd.get_dummies(track_feature_copy[feature], prefix=feature)], axis=1)
    track_feature_copy = track_feature_copy.drop(feature, axis=1)

In [94]:
track_feature_copy.head()

,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,organism,speechiness,tempo,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7,mode_major,mode_minor
0,t_2e8f4b71-8a0b-4b9c-b7d8-fb5208e87f9f,0.167239,0.304348,0.958289,0.719233,0.366513,0.337232,0.440801,0.110348,0.238851,0.865473,6.533884e-01,0.0,0.769258,0.647393,0.199170,0.775905,0.034969,0.401501,0.8,0.223398,0.146012,-0.706908,0.259496,0.481157,0.238427,-0.098389,-0.254960,-0.227383,1,0
1,t_dae2ec0e-ec7b-4b3e-b60c-4a884d0eccb0,0.066561,0.188406,0.727204,0.843003,0.362229,0.395253,0.508696,0.130109,0.420482,0.856651,3.941564e-09,0.0,0.085844,0.734892,0.357639,0.763381,0.051367,0.565365,0.8,0.484707,0.039554,-0.539554,0.105141,0.692589,0.226047,-0.468162,0.164389,-0.769024,1,0
2,t_cf0164dd-1531-4399-bfa6-dec19cd1fedc,0.045423,0.347826,0.962039,0.054904,0.495025,0.597579,0.553525,0.177940,0.842951,0.820145,1.041599e-01,0.0,0.407325,0.764843,0.304721,0.503674,0.073928,0.555589,0.8,0.818449,0.083863,-0.242108,-0.014258,0.096396,0.417641,-0.050576,-0.204757,-0.172563,1,0
3,t_0f90acc7-d5c5-4e53-901d-55610fbd090c,0.117318,0.550725,0.679683,0.042786,0.389652,0.364040,0.586960,0.115343,0.665409,0.811201,1.444968e-05,0.0,0.251502,0.721856,0.702948,0.216724,0.030317,0.532584,0.8,0.594835,0.192498,0.340039,0.034846,-0.389794,0.518381,0.185008,-0.079907,-0.016978,1,0
4,t_36b9ad02-095a-443d-a697-6c7285d9410a,0.081694,0.536232,0.790589,0.251037,0.513665,0.492189,0.636491,0.136824,0.408722,0.868352,5.266899e-01,0.0,0.218370,0.696899,0.888889,0.197565,0.033153,0.608880,0.8,0.591295,0.270586,-0.411061,0.165898,0.225652,0.335518,-0.036643,-0.016300,-0.446870,1,0


In [95]:
track_feature_copy.describe()

,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,organism,speechiness,tempo,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7,mode_major,mode_minor
count,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06,3.706388e+06
mean,1.147970e-01,8.588586e-01,5.573940e-01,3.484361e-01,4.617694e-01,4.857783e-01,5.577719e-01,1.491694e-01,5.918585e-01,8.507039e-01,2.070062e-01,4.793122e-01,2.126515e-01,7.603925e-01,4.996207e-01,4.717471e-01,1.076995e-01,4.802984e-01,7.749120e-01,4.814812e-01,-1.044815e-01,1.553606e-02,5.953425e-02,5.320778e-02,2.273738e-02,6.459076e-04,-2.493560e-02,-1.594925e-03,6.514798e-01,3.485202e-01
std,6.124348e-02,1.598969e-01,2.735567e-01,3.440903e-01,1.717675e-01,2.017956e-01,1.862525e-01,5.094351e-02,2.602857e-01,4.728274e-02,3.447917e-01,3.240019e-01,1.924461e-01,8.649799e-02,2.561809e-01,2.562726e-01,1.427992e-01,1.217166e-01,9.703794e-02,2.725169e-01,4.323750e-01,3.607428e-01,2.804066e-01,3.821163e-01,4.107566e-01,2.118570e-01,3.742747e-01,3.871178e-01,4.765018e-01,4.765018e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.174042e+00,-1.185975e+00,-8.858698e-01,-8.994451e-01,-1.081056e+00,-1.063290e+00,-8.022487e-01,-1.178501e+00,0.000000e+00,0.000000e+00
25%,8.290730e-02,8.115942e-01,3.381824e-01,2.678941e-02,3.316685e-01,3.251607e-01,4.336491e-01,1.106562e-01,4.098384e-01,8.298850e-01,2.465414e-07,1.818182e-01,9.680310e-02,7.286307e-01,2.692308e-01,2.586370e-01,3.737192e-02,3.841560e-01,8.000000e-01,2.519568e-01,-4.085117e-01,-1.757177e-01,-1.471524e-01,-2.348342e-01,-3.122829e-01,-9.018810e-02,-3.155564e-01,-2.704680e-01,0.000000e+00,0.000000e+00
50%,1.061654e-01,9.130435e-01,5.754166e-01,2.214438e-01,4.593377e-01,4.862700e-01,5.740031e-01,1.429287e-01,6.261249e-01,8.583839e-01,2.161882e-04,4.545455e-01,1.294195e-01,7.833421e-01,4.854369e-01,4.646284e-01,5.164614e-02,4.798169e-01,8.000000e-01,4.736072e-01,-4.000132e-02,8.460746e-02,1.291028e-01,6.207571e-02,-1.709655e-02,4.277601e-02,-9.838638e-02,-4.027091e-03,1.000000e+00,0.000000e+00
75%,1.339959e-01,9.710145e-01,7.921223e-01,6.555982e-01,5.851840e-01,6.427212e-01,6.991235e-01,1.801244e-01,8.073180e-01,8.812900e-01,3.132874e-01,7.272727e-01,2.700362e-01,8.179404e-01,7.173145e-01,6.618292e-01,1.046133e-01,5.594691e-01,8.000000e-01,7.050109e-01,1.915962e-01,2.832876e-01,2.746523e-01,2.832823e-01,3.724048e-01,1.476096e-01,2.058770e-01,2.218316e-01,1.000000e+00,1.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000457e+00,8.624035e-01,6.509755e-01,1.202726e+00,1.000955e+00,4.130149e-01,1.092165e+00,1.204228e+00,1.000000e+00,1.000000e+00


In [87]:
track_feature_copy['mode'].drop_duplicates().sort_values()

0        major
14190    minor
Name: mode, dtype: object

In [88]:
track_feature_copy['mode'] = track_feature_copy['mode'].map(categorical_columns_map['mode'])

In [89]:
track_feature_copy['mode'].drop_duplicates().sort_values()

0        0
14190    1
Name: mode, dtype: int64